 # **단순 방법(예시 코드 참고)**

- 노래마다 tag들의 개수를 구한다. ex - 노래 {tag1 : 3, tag2 : 2, tag3 : 4}
- tag3이 제일 높은 애를 가지므로 tag3이 속한 감정을 label로 잡음(ex - 우울한)
- nlp label에서 우울한이 output으로 나왔을 때, 위에서 우울한으로 매칭되는 플레이리스트or노래들을 추천해주는 형태

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/song_tag_gnr_map.csv')
df

,dtl_gnr,song_name,gnr,song_id,tags
0,GN0901,Feelings,GN0900,0,"비오는날, 드라이브"
1,GN0901,Feelings,GN0900,0,"비오는날, 추억, 회상"
2,"GN1601, GN1606","Bach : Partita No. 4 In D Major, BWV 828 - II....",GN1600,1,NaN
3,GN0901,Solsbury Hill (Remastered 2002),GN0900,2,NaN
4,"GN1102, GN1101",Feeling Right (Everything Is Nice) (Feat. Popc...,GN1100,3,1
...,...,...,...,...,...
5378713,"GN0105, GN0101",스치듯 안녕,GN0100,707986,윤종신
5378714,"GN0105, GN0101",스치듯 안녕,GN0100,707986,"윤종신, 명곡, 김광석, 발라드"
5378715,"GN0105, GN0101",스치듯 안녕,GN0100,707986,"잔잔한, 이별, 발라드, 밤, 새벽"
5378716,"GN1807, GN1801",숲의 빛,GN1800,707987,"집중, 노동요, 새벽, 오후, 자장가, 공부"


## song id - tag 매핑

In [2]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5378718 entries, 0 to 5378717
Data columns (total 5 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   dtl_gnr    object
 1   song_name  object
 2   gnr        object
 3   song_id    int64 
 4   tags       object
dtypes: int64(1), object(4)
memory usage: 205.2+ MB


In [4]:
# tag 없는 노래 제거
df = df[df['tags'].notnull()]
# tag를 song_id 기준으로 묶음
df[['tags', 'song_id']].groupby('song_id').sum() 

,tags
song_id,
0,"비오는날, 드라이브비오는날, 추억, 회상"
3,"1감성, 듣기좋은, 드라이브, Pop, 여행, 기분전환, 팝, 신나는클럽, 매장음악..."
4,뉴에이지뉴에이지
5,"비오는날재즈락발라드휴식, 리메이크, 재즈, 힐링송, 힐링, 기분전환, 기분_전환, 음악"
6,"클래식, 산책, 조깅"
...,...
707984,월드뮤직
707985,"Pop해외차트, 시대별차트신디로퍼, Pop, 1980, 라이벌록"
707986,"힙합, 랩발라드감성, 리메이크곡, 카페, 잔잔한, 추억, 위로, 힐링, 사랑, 이별..."


In [14]:
# tag를 sum하면 두개 단어가 붙어 나와서 tag 마지막에 쉼표 추가하고 다시 도전
# index 재설정
df = df.reset_index()
df.drop('index', axis = 1, inplace = True)
df['tags'] = [df['tags'][i] + ', ' for i in range(len(df))] 
df[['tags', 'song_id']].groupby('song_id').sum() 

,tags
song_id,
0,"비오는날, 드라이브, 비오는날, 추억, 회상,"
3,"1, 감성, 듣기좋은, 드라이브, Pop, 여행, 기분전환, 팝, 신나는, 클럽, ..."
4,"뉴에이지, 뉴에이지,"
5,"비오는날, 재즈, 락, 발라드, 휴식, 리메이크, 재즈, 힐링송, 힐링, 기분전환,..."
6,"클래식, 산책, 조깅,"
...,...
707984,"월드뮤직,"
707985,"Pop, 해외차트, 시대별차트, 신디로퍼, Pop, 1980, 라이벌, 록,"
707986,"힙합, 랩, 발라드, 감성, 리메이크곡, 카페, 잔잔한, 추억, 위로, 힐링, 사랑..."


In [16]:
song_tag = df[['tags', 'song_id']].groupby('song_id').sum().reset_index()
song_tag

,song_id,tags
0,0,"비오는날, 드라이브, 비오는날, 추억, 회상,"
1,3,"1, 감성, 듣기좋은, 드라이브, Pop, 여행, 기분전환, 팝, 신나는, 클럽, ..."
2,4,"뉴에이지, 뉴에이지,"
3,5,"비오는날, 재즈, 락, 발라드, 휴식, 리메이크, 재즈, 힐링송, 힐링, 기분전환,..."
4,6,"클래식, 산책, 조깅,"
...,...,...
615137,707984,"월드뮤직,"
615138,707985,"Pop, 해외차트, 시대별차트, 신디로퍼, Pop, 1980, 라이벌, 록,"
615139,707986,"힙합, 랩, 발라드, 감성, 리메이크곡, 카페, 잔잔한, 추억, 위로, 힐링, 사랑..."
615140,707987,"집중, 노동요, 새벽, 오후, 자장가, 공부,"


In [18]:
song_tag['tags'] = [song_tag['tags'][i][:-2] for i in range(len(song_tag))]
song_tag

,song_id,tags
0,0,"비오는날, 드라이브, 비오는날, 추억, 회상"
1,3,"1, 감성, 듣기좋은, 드라이브, Pop, 여행, 기분전환, 팝, 신나는, 클럽, ..."
2,4,"뉴에이지, 뉴에이지"
3,5,"비오는날, 재즈, 락, 발라드, 휴식, 리메이크, 재즈, 힐링송, 힐링, 기분전환,..."
4,6,"클래식, 산책, 조깅"
...,...,...
615137,707984,월드뮤직
615138,707985,"Pop, 해외차트, 시대별차트, 신디로퍼, Pop, 1980, 라이벌, 록"
615139,707986,"힙합, 랩, 발라드, 감성, 리메이크곡, 카페, 잔잔한, 추억, 위로, 힐링, 사랑..."
615140,707987,"집중, 노동요, 새벽, 오후, 자장가, 공부"


## song id 별로 어떤 tag 몇 번 나왔는지 확인

In [29]:
from collections import Counter
a = song_tag['tags'][0].split(', ')
cnt = dict(Counter(a))
cnt

{'비오는날': 2, '드라이브': 1, '추억': 1, '회상': 1}

In [31]:
dict(Counter(song_tag['tags'][0].split(', '))) 

{'비오는날': 2, '드라이브': 1, '추억': 1, '회상': 1}

In [34]:
song_tag['tag_count'] =    [dict(Counter(song_tag['tags'][i].split(', '))) for i in range(len(song_tag))]

In [35]:
song_tag

,song_id,tags,tag_count
0,0,"비오는날, 드라이브, 비오는날, 추억, 회상","{'비오는날': 2, '드라이브': 1, '추억': 1, '회상': 1}"
1,3,"1, 감성, 듣기좋은, 드라이브, Pop, 여행, 기분전환, 팝, 신나는, 클럽, ...","{'1': 1, '감성': 1, '듣기좋은': 1, '드라이브': 2, 'Pop':..."
2,4,"뉴에이지, 뉴에이지",{'뉴에이지': 2}
3,5,"비오는날, 재즈, 락, 발라드, 휴식, 리메이크, 재즈, 힐링송, 힐링, 기분전환,...","{'비오는날': 1, '재즈': 2, '락': 1, '발라드': 1, '휴식': 1..."
4,6,"클래식, 산책, 조깅","{'클래식': 1, '산책': 1, '조깅': 1}"
...,...,...,...
615137,707984,월드뮤직,{'월드뮤직': 1}
615138,707985,"Pop, 해외차트, 시대별차트, 신디로퍼, Pop, 1980, 라이벌, 록","{'Pop': 2, '해외차트': 1, '시대별차트': 1, '신디로퍼': 1, '..."
615139,707986,"힙합, 랩, 발라드, 감성, 리메이크곡, 카페, 잔잔한, 추억, 위로, 힐링, 사랑...","{'힙합': 1, '랩': 1, '발라드': 5, '감성': 3, '리메이크곡': ..."
615140,707987,"집중, 노동요, 새벽, 오후, 자장가, 공부","{'집중': 1, '노동요': 1, '새벽': 1, '오후': 1, '자장가': 1..."


In [43]:
song_tag['tag_count'][0]

{'비오는날': 2, '드라이브': 1, '추억': 1, '회상': 1}

In [45]:
[k for k,v in song_tag['tag_count'][0].items() if max(song_tag['tag_count'][0].values()) == v]

['비오는날']

In [62]:
song_tag['max_key'] = [k for i in range(len(song_tag)) for k,v in song_tag['tag_count'][i].items() if max(song_tag['tag_count'][i].values()) == v]

SyntaxError: invalid syntax (<ipython-input-62-a44d5649aea0>, line 1)

In [64]:
max_key = []
for i in range(len(song_tag)):
    di = song_tag['tag_count'][i]
    max_key.append([k for k,v in di.items() if max(di.values()) == v])

In [65]:
max_key

[['비오는날'],
 ['기분전환', '신나는'],
 ['뉴에이지'],
 ['재즈'],
 ['클래식', '산책', '조깅'],
 ['클래식', '코로나'],
 ['발라드'],
 ['Lofi'],
 ['감성', 'Pop', '로맨틱'],
 ['재즈', '가을밤', '로맨틱', '빅밴드', '스윙'],
 ['일렉', 'EDM'],
 ['좋은곡', '스트레스', '장르불문', '카카오톡', '기분전환', '일상', '오픈채팅', '트렌디', '팝', '신나는'],
 ['마마무'],
 ['스트레스', '해소음악'],
 ['잔잔한'],
 ['클래식'],
 ['먹자', '놀자'],
 ['해외차트', '시대별차트', '팝', '추억', '1960', 'Rock'],
 ['연말', '겨울', '클래식', '훈훈', '크리스마스', '캐롤', '추위'],
 ['일렉'],
 ['팝'],
 ['2월',
  '신곡',
  '에이셉퍼그',
  '케샤',
  '코스모스미드나잇',
  '루이톰린슨',
  '메디슨비어',
  'Pop',
  '가요',
  '메간트레이너',
  '일본',
  '도쿄',
  '시부야',
  'JPOP',
  '제이팝'],
 ['카페', '프랑스'],
 ['신나는'],
 ['뉴에이지'],
 ['감성', 'OST'],
 ['아이돌', '인디음악', '트로트', '팝송', '사무실', '동요'],
 ['스트레스', '일렉'],
 ['인디포크', '인디', '감성'],
 ['추억', 'JPOP', '애니메이션OST', 'OST'],
 ['띵곡', '페이버릿', '명곡', '내적댄스유발'],
 ['카페',
  '취향저격',
  '팝',
  '음색',
  '여자보컬',
  '매력적인',
  '기분전환',
  '설렘',
  '감미로운',
  '고막여친',
  '분위기의',
  '밤',
  '알앤비',
  '귀르가즘',
  'RnB',
  '감각적인',
  '섹시한',
  '그루브한',
  '음색깡패'],
 ['팝'],
 ['겨울', '잔잔한', '소울', '알앤비', '

In [66]:
song_tag['max_key'] = max_key
song_tag

,song_id,tags,tag_count,max_key
0,0,"비오는날, 드라이브, 비오는날, 추억, 회상","{'비오는날': 2, '드라이브': 1, '추억': 1, '회상': 1}",[비오는날]
1,3,"1, 감성, 듣기좋은, 드라이브, Pop, 여행, 기분전환, 팝, 신나는, 클럽, ...","{'1': 1, '감성': 1, '듣기좋은': 1, '드라이브': 2, 'Pop':...","[기분전환, 신나는]"
2,4,"뉴에이지, 뉴에이지",{'뉴에이지': 2},[뉴에이지]
3,5,"비오는날, 재즈, 락, 발라드, 휴식, 리메이크, 재즈, 힐링송, 힐링, 기분전환,...","{'비오는날': 1, '재즈': 2, '락': 1, '발라드': 1, '휴식': 1...",[재즈]
4,6,"클래식, 산책, 조깅","{'클래식': 1, '산책': 1, '조깅': 1}","[클래식, 산책, 조깅]"
...,...,...,...,...
615137,707984,월드뮤직,{'월드뮤직': 1},[월드뮤직]
615138,707985,"Pop, 해외차트, 시대별차트, 신디로퍼, Pop, 1980, 라이벌, 록","{'Pop': 2, '해외차트': 1, '시대별차트': 1, '신디로퍼': 1, '...",[Pop]
615139,707986,"힙합, 랩, 발라드, 감성, 리메이크곡, 카페, 잔잔한, 추억, 위로, 힐링, 사랑...","{'힙합': 1, '랩': 1, '발라드': 5, '감성': 3, '리메이크곡': ...",[발라드]
615140,707987,"집중, 노동요, 새벽, 오후, 자장가, 공부","{'집중': 1, '노동요': 1, '새벽': 1, '오후': 1, '자장가': 1...","[집중, 노동요, 새벽, 오후, 자장가, 공부]"


In [67]:
song_tag.to_csv('./data/song_tag_max.csv', index = False)